In [1]:
import telegram, logging

from telegram.ext import Updater, CommandHandler, Filters, MessageHandler

from chatbotwrapper import PythonChatBot

In [2]:
chatbot = PythonChatBot(threshold=0.5)

In [3]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

In [4]:
bot_train = telegram.Bot(token='386950793:AAFWQO5Exopgk-WwzHcrKEArN7tVM20OL4A')

updater_train = Updater(token='386950793:AAFWQO5Exopgk-WwzHcrKEArN7tVM20OL4A')

dispatcher_train = updater_train.dispatcher

In [5]:
def start_train(bot, update):
    bot_train.send_message(chat_id=update.message.chat_id, text="""Привет! Я готов обучаться\n\
    Пришли сообщением вопрос и ответ на него.\n\
    Не забудь разделить их вот этим симоволом "|"
""")

In [6]:
def train(bot, update):
    user = update.message.from_user
    text = update.message.text
    logging.info("text of %s: %s" % (user.first_name, text))
    bot.send_message(chat_id=update.message.chat_id, text="""
    Спасибо, я постараюсь хорошо обучиться.\n
    Покидай обучалок еще, мне нравится учиться
    """)
    try:
        train_text = text.split('|')
        chatbot.train(train_text[0], train_text[1])
    except Exception as e:
        logging.warning("Произошла ошибка -{}- \nНа обучающем тексте -{}-".format(e, text))
        bot.send_message(chat_id=update.message.chat_id, text="Не получилось обучиться. Может быть неправильный формат?")
    else:
        logging.info("Обучение на обучающем тексте -{}- прошло успешно".format(text))

In [7]:
start_handler_train = CommandHandler('start', start_train)
text_handler_train = MessageHandler(Filters.text, train)

dispatcher_train.add_handler(start_handler_train)
dispatcher_train.add_handler(text_handler_train)

In [8]:
updater_train.start_polling()

In [9]:
bot_predict = telegram.Bot(token='383933408:AAHmUeE2FX_tODZfiqZUSDMNHYuO-31q1p4')

updater_predict = Updater(token='383933408:AAHmUeE2FX_tODZfiqZUSDMNHYuO-31q1p4')

dispatcher_predict = updater_predict.dispatcher

In [10]:
def start_predict(bot, update):
    bot_predict.send_message(chat_id=update.message.chat_id, text="""Привет! Меня еще учат, но кажется\
    что я более-менее готов отвечать на вопросы""")

In [11]:
def predict(bot, update):
    user = update.message.from_user
    text = update.message.text
    logging.info("text of %s: %s" % (user.first_name, text))
    try:
        response = chatbot.response(text)
    except Exception as e:
        bot.send_message(chat_id=update.message.chat_id, text="К сожалению что-то пошло не так")
        logging.warning("Произошла ошибка -{}- \nПытался ответить на -{}-".format(e, text))
    else:
        bot.send_message(chat_id=update.message.chat_id, text=response)
        logging.info("Ответа на вопрос -{}- прошел успешно".format(text))

In [12]:
def predict_sound(bot, update):
    user = update.message.from_user
    filepath = './{}_question.wav'.format(user.first_name)
    try:
        file_id = message.voice.file_id
        newFile = bot.get_file(file_id)
        newFile.download(filepath)
        return chatbot.response_sound(filepath)
    except Exception as e:
        return ("Exception: "+str(e))


In [13]:
start_handler_predict = CommandHandler('start', start_predict)
text_handler_predict = MessageHandler(Filters.text & (~Filters.voice), predict)
voice_handler_predict = MessageHandler(Filters.voice, predict_sound)

dispatcher_predict.add_handler(start_handler_predict)
dispatcher_predict.add_handler(text_handler_predict)
dispatcher_predict.add_handler(voice_handler_predict)

In [14]:
updater_predict.start_polling()

2017-09-16 18:16:42,795 - root - INFO - text of Maxim: Как дела?
2017-09-16 18:16:42,870 - chatterbot.adapters - INFO - Recieved input statement: Как дела?
2017-09-16 18:16:42,906 - chatterbot.adapters - INFO - "Как дела?" is a known statement
2017-09-16 18:16:42,974 - chatterbot.adapters - INFO - Using "Как дела?" as a close match to "как дела?"
2017-09-16 18:16:42,978 - chatterbot.adapters - INFO - Selecting response from 1 optimal responses.
2017-09-16 18:16:42,979 - chatterbot.response_selection - INFO - Selecting first response from list of 1 options.
2017-09-16 18:16:42,980 - chatterbot.adapters - INFO - Response selected. Using " Да нормально"
2017-09-16 18:16:42,982 - chatterbot.adapters - INFO - BestMatch selected " Да нормально" as a response with a confidence of 1.0
2017-09-16 18:16:43,060 - chatterbot.adapters - INFO - LowConfidenceAdapter selected "Я недоучился, поэтому не могу понять о чем речь" as a response with a confidence of 0
2017-09-16 18:16:43,064 - chatterbot.ada

List Trainer: [####################] 100%

2017-09-16 18:17:48,854 - root - INFO - Обучение на обучающем тексте -Тыщ пышь тырыпышь | Пыжыдышь- прошло успешно


2017-09-16 18:17:53,000 - root - INFO - text of Maxim: Тыщ пышь тырыпышь
2017-09-16 18:17:53,072 - chatterbot.adapters - INFO - Recieved input statement: Тыщ пышь тырыпышь
2017-09-16 18:17:53,077 - chatterbot.adapters - INFO - "Тыщ пышь тырыпышь" is not a known statement
2017-09-16 18:17:53,157 - chatterbot.adapters - INFO - Using "Тыщ пышь тырыпышь" as a close match to "тыща пыхать тырыпыхать"
2017-09-16 18:17:53,161 - chatterbot.adapters - INFO - Selecting response from 1 optimal responses.
2017-09-16 18:17:53,163 - chatterbot.response_selection - INFO - Selecting first response from list of 1 options.
2017-09-16 18:17:53,167 - chatterbot.adapters - INFO - Response selected. Using " Пыжыдышь"
2017-09-16 18:17:53,169 - chatterbot.adapters - INFO - BestMatch selected " Пыжыдышь" as a response with a confidence of 0.77
2017-09-16 18:17:53,246 - chatterbot.adapters - INFO - LowConfidenceAdapter selected "Я недоучился, поэтому не могу понять о чем речь" as a response with a confidence of 